# Week 1

In [4]:
# Imports
import torch

def sampler(n = 300):
    mu = torch.zeros(2)
    cov = torch.tensor([[2.0, 0.5], [0.5, 4.0]])
    x = torch.distributions.multivariate_normal.MultivariateNormal(mu, cov).rsample([n])
    
    w_transp = torch.tensor([[-1.7], [1.2]])
    
    y = torch.distributions.bernoulli.Bernoulli(1 / (1 + torch.exp(-torch.matmul(x, w_transp)))).sample()
    
    return x, y

n = 300
x, y = sampler(n)

train_x, test_x = x[:int(0.8*n)], x[int(0.8*n):]
train_y, test_y = y[:int(0.8*n),], y[int(0.8*n):,]

def train_loop(x_train, y_train, model, loss_fn, optimizer, epochs = 10000):
    size = len(x_train)
    for _ in range(epochs):
        # Compute prediction and loss
        pred = model(x_train)
        loss = loss_fn(pred, y_train)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test_loop(x_test, y_test, model, loss_fn):
    with torch.no_grad():
        preds = model(x_test)
        loss_test = loss_fn(preds, y_test)
        print(f"Final loss in the test set: {loss_test.item()}")

input_dim = 2
output_dim = 1
learning_rate = 5e-3

model = torch.nn.Linear(input_dim, output_dim, bias=False)

loss_fn = torch.nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 20000

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_x = train_x.to(device)
train_y = train_y.to(device)
model = model.to(device)

train_loop(train_x, train_y, model, loss_fn, optimizer, epochs)

model = model.to(torch.device("cpu"))
test_loop(test_x, test_y, model, loss_fn)
print(model.weight)

Final loss in the test set: 0.3889238238334656
Parameter containing:
tensor([[-1.7263,  1.2110]], requires_grad=True)
